## User Management (Keycloak)
Interaction with EOEPCA endpoint using python Demo Client.

In [ ]:
import jwt
import json
import requests
from getpass import getpass
from keycloak import KeycloakPostError
from identityutils.keycloak_client import KeycloakClient
import requests

base_domain = "develop.eoepca.org"
platform_domain = f"identity.keycloak.{base_domain}"
server_url = f"https://{platform_domain}"
realm = "master"
dummy_service_url = f"https://identity.dummy-service.{base_domain}"
identity_api_url = f"https://identity.api.{base_domain}"

## Client
We instantiate an object to interact with the Keycloak.

In [ ]:
admin_password = getpass("Admin password: ")
keycloak = KeycloakClient(
    server_url=server_url,
    realm=realm,
    username="admin",
    password=admin_password,
)

## User Authentication
User authenticates and the client receives an ID Token (JWT) that represents the user, and is used to identify the user in UMA authorization flows.

#### Create Users
Create two users, an Eric and Alice both with user role.

In [ ]:
eric_id = keycloak.create_user("eric", "eric", ["user"])
print("Created Eric user with id: " + eric_id)
alice_id = keycloak.create_user("alice", "alice", ["user"])
print("Created Alice user with id: " + alice_id)

#### Inspect Eric User Token

In [ ]:
token = keycloak.get_user_token("eric", "eric")
print("Eric token:\n" + json.dumps(token, indent = 2))
eric_access_token = token["access_token"]
jwt_header = jwt.get_unverified_header(eric_access_token)
print("JWT Header:\n" + json.dumps(jwt_header, indent = 2))
jwt_payload = jwt.decode(eric_access_token, options={"verify_signature": False})
print("JWT Payload:\n" + json.dumps(jwt_payload, indent = 2))

#### Inspect Alice User Token

In [ ]:
token = keycloak.get_user_token("alice", "alice")
print("Eric token:\n" + json.dumps(token, indent = 2))
alice_access_token = token["access_token"]
jwt_header = jwt.get_unverified_header(alice_access_token)
print("JWT Header:\n" + json.dumps(jwt_header, indent = 2))
jwt_payload = jwt.decode(alice_access_token, options={"verify_signature": False})
print("JWT Payload:\n" + json.dumps(jwt_payload, indent = 2))

The ID Token (JWT) identifies the user via user_name / sub (Subject) fields, and the client via the aud (Audience) field. The JWT is signed and can be verified, using the kid (Key ID) field, via the JWKS endpoint of the Authorization Server.

## Protect resources
Access a protected resources using UMA flow.

#### Assign premium role to Eric
Roles are used to define policies that will protect resources based on roles - Role-based access control (RBAC)

In [ ]:
realm_role = keycloak.create_realm_role('user-premium')
print("Created realm role: " + realm_role)
keycloak.assign_realm_roles_to_user(eric_id, realm_role)
print("Assigned " + realm_role + " role to Eric")

#### Register client
Register demo client

In [ ]:
client_id = "demo"
client_secret= "demo"
client_payload = {
    "clientId": client_id,
    "clientSecret": client_secret,
    "name": "Demo client",
    "description": "Client used on demo notebook"
}
keycloak.create_client(client_payload, skip_exists=True)
print("Created client: demo")

## Protect URIs
Right now, resources are protected by a default policy, which grants access to users within the realm.
Let's see how Keycloak protects resources using role based and user based policies.

#### Register resources
Register

In [ ]:
resources = [
    {
        "name": "Premium resource",
        "uri": "/protected/premium/*"
    },
    {
        "name": "Eric space",
        "uri": "/eric/*"
    },
    {
        "name": "Alice space",
        "uri": "/alice/*"
    }
]
keycloak.register_resources(client_id, resources, skip_exists=True)

#### Register policies
Register role based and user based policies

In [ ]:
policy = {
    "name": 'Only Premium User Policy',
    "roles": [
        {
            "id": "user-premium"
        }
    ]
}
p = keycloak.register_role_policy(client_id, policy, skip_exists=True)
print("Only Premium User Policy:")
print(p)

policy = {
    "name": 'Only Eric User Policy',
    "users": [eric_id]
}
p = keycloak.register_user_policy(client_id, policy)
print("Only Eric User Policy:")
print(p)

policy = {
    "name": 'Only Alice User Policy',
    "users": [alice_id]
}
p = keycloak.register_user_policy(client_id, policy)
print("Only Alice User Policy:")
print(p)

#### Register resource permissions
Resources permissions are set by assigning policies to resources.

In [ ]:
permissions = [
    {
        "name": "Premium permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Premium Resource"
        ],
        "policies": [
            "Only Premium User Policy"
        ]
    },
    {
        "name": "Eric space permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Protected Resource"
        ],
        "policies": [
            "Only Eric User Policy"
        ]
    },
    {
        "name": "Alice space permission",
        "type": "resource",
        "logic": "POSITIVE",
        "decisionStrategy": "UNANIMOUS",
        "resources": [
            "Protected Resource"
        ],
        "policies": [
            "Only Alice User Policy"
        ]
    }
]
r = keycloak.assign_resources_permissions(client_id, permissions, skip_exists=True)
print(r)

#### Get PAT (Protection API token)
PAT (Protection API token) is used to access Keycloak's Protection API, which manages resources and policies.

In [ ]:
pat = keycloak.generate_protection_pat(client_id, client_secret)
print(json.dumps(pat, indent=2))
access_token = pat['access_token']

### Get Resource Id
Get Premium Resource id

In [ ]:
# it's possible to query resources by many fields, including name and uri
premium_resource_id = keycloak.get_resource_id(name="Premium Resource")[0]
print("Premium resource: " + premium_resource_id)
eric_resource_id = keycloak.get_resource_id(uri="/eric/*")[0]
print("Eric resource: " + eric_resource_id)
alice_resource_id = keycloak.get_resource_id(uri="/alice/*")[0]
print("Alice resource: " + alice_resource_id)

#### Get UMA access token for premium resource for both Eric and Alice

In [ ]:
uma_ticket = keycloak.create_permission_ticket(resources=[eric_resource_id])['ticket']
print('UMA ticket for resource ' + eric_resource_id + ':\n' + uma_ticket)
eric_access_token = keycloak.get_user_token("eric", "eric")['access_token']
eric_rpt = keycloak.get_rpt(eric_access_token, uma_ticket)['access_token']
print('Eric RPT:\n' + str(eric_rpt))

In [ ]:
uma_ticket = keycloak.create_permission_ticket(resources=[eric_resource_id])['ticket']
print('UMA ticket for resource ' + eric_resource_id + ':\n' + uma_ticket)
alice_access_token = keycloak.get_user_token("alice", "alice")['access_token']
try:
    alice_uma_access_token = keycloak.get_rpt(alice_access_token, uma_ticket)['access_token']
except KeycloakPostError as e:
    print(str(e))

Trying to get a UMA token for Alice results in a 403 Forbidden Error. The reason being Alice is not allowed to access the `/eric/*` resource because it's protected for only `eric` user.

#### Use Eric UMA access token to access the eric space resource

In [ ]:
headers = {
    "cache-control": "no-cache",
    "Authorization": "Bearer " + eric_rpt
}
url = dummy_service_url + "/eric"
print('GET ' + url)
response = requests.get(url, headers=headers, verify=False)
print(str(response.status_code))

The expected result is 200 meaning a successfuly access to the eric space resource by Eric.

#### Identity API Get Resources

In [ ]:
import requests

resources_url = identity_api_url + "/" + client_id + "/resources"
print("GET " + resources_url)
response = requests.get(resources_url)
try:
    print(response.json())
except:
    print(response)

#### Identity API Get resource by id

In [ ]:
response = requests.get(resources_url)
resource_id = response.json()[0]["_id"]
print(resource_id)
response = requests.get(resources_url + "/" + resource_id)
print(response.text)

#### Identity API Register resource

In [ ]:
data = {
    'attributes': {},
    'name': 'A resource',
    'ownerManagedAccess': False,
    'scopes': ["access"],
    'uris': ["/protected/*"]
}
response = requests.post(resources_url, json=data)
print("Response code: ", str(response.status_code))
print("Resource added: ", response.json())

#### Identity API Update resource

In [ ]:
resource_id = keycloak.get_resource_id(name="A resource")[0]
data = {
    "attributes": {},
    "name": "A resource",
    "ownerManagedAccess": True,
    "uris": ["/protected/*"]
}
response = requests.put(resources_url + "/" + resource_id, json=data)
print("Response code: ", str(response.status_code))

#### Identity API Delete resource

In [ ]:
# get resource_id from the get_resources endpoint
resource_id = keycloak.get_resource_id(name="A resource")[0]
response = requests.delete(resources_url + "/" + resource_id)
print("Response code: ", str(response.status_code))

#### Identity API Get client Policies

In [ ]:
policies_url = identity_api_url + "/" + client_id + "/policies"
response = requests.get(policies_url)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API create client Policy

In [ ]:
data = {
    "type": "client",
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "My Policy"
}
response = requests.post(policies_url + "/client", json=data)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API Create Aggregated policy

In [ ]:
data = {
    "type": "aggregate",
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Aggregated policy",
    "policies": ["My Policy"],
    "description": "Policy description"
}
response = requests.post(policies_url + "/aggregated", json=data)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API Create scope policy

In [ ]:
data = {
    "type": "client-scope",
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "My Policy"
}
response = requests.post(policies_url + "/scope", json=data)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API Create group policy

In [ ]:
data = {
    "type": "group",
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Group policy",
    "groups": [{"id": "group", "path": "group"}],
    "groupsClaim": "Groups claim",
    "description": "description"
}
response = requests.post(policies_url + "/group", json=data)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API - Create regex Policy

In [ ]:
data = {
    "type": "regex",
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Regex policy",
    "pattern": ".*",
    "targetClaim": "preferred_username",
    "description": "Match all usernames"
}
response = requests.post(policies_url + "/regex", json=data)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API Create role policy

In [ ]:
data = {
    "type": "role",
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Role policy",
    "roles": [
        {
            "id": "b80601c3-9e0f-40fd-a03c-34cd759bfc63",
            "required": False
        }
    ]
}
response = requests.post(policies_url + "/role", json=data)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API create time policy

In [ ]:
data = {
    "type": "time",
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Time policy",
    "description": "description",
    "year": 2023
}
response = requests.post(policies_url + "/time", json=data)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API Create user policy

In [ ]:
data = {
    "type": "user",
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "User policy",
    "users": eric_id,
    "description": "description"
}
response = requests.post(policies_url + "/user", json=data)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API Update policies

In [ ]:
policy_id = "8e90c31f-759a-4679-86c5-56cad118f12e"
data = {
    "type": "user",
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "User policy",
    "description": "description"
}
response = requests.put(policies_url + "/" + policy_id, json=data)
print("Response code: ", str(response.status_code))

#### Identity API delete policies

In [ ]:
policy_id = "8e90c31f-759a-4679-86c5-56cad118f12e"
response = requests.put(policies_url + "/" + policy_id)
print("Response code: ", str(response.status_code))

#### Identity API get client permissions

In [ ]:
client_permissions_url = identity_api_url + "/" + client_id + "/permissions"
response = requests.get(client_permissions_url)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API get client management permissions

In [ ]:
response = requests.get(client_permissions_url + "/management")
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())


#### Identity API get client resources permissions

In [ ]:
response = requests.get(client_permissions_url + "/resources")
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())


#### Identity API create client resources permissions

In [ ]:
data = {
    "type": "resource",
    "logic": "POSITIVE",
    "decisionStrategy": "UNANIMOUS",
    "name": "Permission-Name 2",
    "resources": [
        "5bd655ec-2575-406e-aa08-28b1bd25f476"
    ],
    "policies": [
        "57d4a363-6b40-4dec-93e9-a46a1a8e492f"
    ]
}
response = requests.get(client_permissions_url + "/resources", json=data)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API update client management permissions

In [ ]:
data = {
    "enabled": False
}
response = requests.put(client_permissions_url + "/management", json=data)
print("Response code: ", str(response.status_code))
print("Response text: ", response.json())

#### Identity API update client resources permissions

In [ ]:
permission_id = "2193eed8-93e5-471f-bb67-a3726b333fb1"
update_resources_permissions_url = client_permissions_url + "/resources/" + permission_id
data = {
    "decisionStrategy": "UNANIMOUS",
    "description": "A permission that applies to the default resource type",
    "logic": "POSITIVE",
    "name": "Default Permission",
    "resourceType": "urn:gatekeeper:resources:default",
    "type": "resource"
}
response = requests.put(update_resources_permissions_url, json=data)
print("Response code: ", str(response.status_code))

#### Identity API Register and Protect a Resource

In [ ]:
data = [
    {
        "name": "Eric resource",
        "uris": ["/eric/*"],
        "permissions": {
            "user": ["eric"]
        }
    },
    {
        "name": "Alice resource",
        "uris": ["/alice/*"],
        "permissions": {
            "user": ["alice"]
        }
    }]

response = requests.post(identity_api_url + "/" + client_id + "/resources", json=data)
print("Response code: ", str(response.status_code))

#### Identity API Delete Resource and its policies and permissions

In [ ]:
resource_name = "Eric resource"
response = requests.delete(identity_api_url + "/" + client_id + "/resources/" + resource_name)
print(response.status_code)
print(response.text)
resource_name = "Alice resource"
response = requests.delete(identity_api_url + "/" + client_id + "/resources/" + resource_name)
print(response.status_code)
print(response.text)

#### Identity API Create client, create and protect resources in one endpoint

In [ ]:
payload = {
    "clientId": "dummy-service",
    "name": "Dummy Service",
    "description": "Client used for Dummy service",
    "resources": [
        {
            "name": "Eric space",
            "uris": ["/eric/*"],
            "permissions": {
                "user": ["eric"]
            }
        },
        {
            "name": "Alice space",
            "uris": ["/alice/*"],
            "permissions": {
                "user": ["alice"]
            }
        }
    ]
}
response = requests.post(identity_api_url + '/clients', json=payload)
print(response.status_code)
print(response.text)

#### Execute script to create a client with protected resources

In [ ]:
import subprocess
import urllib.request

with urllib.request.urlopen(
        "https://raw.githubusercontent.com/EOEPCA/um-identity-service/master/scripts/create-client.sh") as f:
    script = f.read().decode('utf-8')
    out = open("../data/create-client.sh", "a")
    out.write(script)
    out.flush()

access_token = keycloak.get_user_token("admin", admin_password)["access_token"]

client_id = input("Client id: ")
client_name = "Dummy service Gatekeeper"
client_description = "Client to be used by Dummy service Gatekeeper"

subprocess.check_call(["../data/create-client.sh",
                       "-e demo",
                       "-t " + access_token,
                       "--id " + client_id,
                       "--name " + client_name,
                       "--description " + client_description,
                       "--resource=\"Eric space\"", "--uris=/eric/*", "--users=eric",
                       "--resource=\"Alice space\"", "--uris=/alice/*", "--users=alice"
                       ], shell=True, text=True)
